## 1. 수정이 필요한 세 함수 내용 확인

In [ ]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt]) # 편향은 초기에 지나친 영향을 주어 학습에 역효과 없도록 0으로 초기화하여 생성
    
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x # 역전파를 위해 이용될 보조 정보로 x도 함께 반환 

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose() # 가중치의 손실 기울기 구하는 과정에서 필요 (80p)
    
    G_w = np.matmul(g_output_w, G_output) # (m, n) @ (n, o) = (m, o)
    G_b = np.sum(G_output, axis = 0) # B_j는 Y_1j, Y_2j, ... Y_mj모두를 통해 L값에 영향을 미침
    
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

## 2. 은닉 계층 하나를 위한 파라미터 생성 함수 정의

In [1]:
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])
    
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

## 3. 은닉 계층 하나를 위한 순전파 함수 정의

In [2]:
def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden
    
    hidden = relu(np.matmul(x, pm_hidden['w'] + pm_hidden['b'])) # (n, m) @ (m, h) + (h,) = (n, h)
    output = np.matmul(hidden, pm_output['w'] + pm_output['b'])  # (n, h) @ (h, o) + (o,) = (n, o)
    
    return output, [x, hidden]

def relu(x):
    return np.maximum(x, 0)

## 4. 은닉 계층 하나를 위한 역전파 함수 정의

In [ ]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    x, hidden = aux
    
    g_output_w_out = hidden.transpose() # (h, n)
    G_w_out = np.matmul(g_output_w_out, G_output) # (h, n) @ (n, o) = (h, o)
    G_b_out = np.sum(G_output, axis = 0) # (o, )
    
    g_output_hidden = pm_output['w'].transpose() # (o, h)
    G_hidden = np.matmul(G_output, g_output_hidden) # (n, o) @ (o, h) = (n, h)
    
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
    G_b_hid = np.sum(G_hidden, axis = 0)
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid
    
def relu_derv(y):
    return np.sign(y)

## 5. 가변적 은닉 계층 구성을 위한 파라미터 생성 함수 정의

In [1]:
def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
        
    pm_output = alloc_param_pair([prev_cnt, output_cnt])

## 6. 가변적 은닉 계층 구성을 위한 순전파 함수 정의

In [2]:
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens # 역전파를 위해 은닉 계층의 입력값들을 hiddens 리스트에 저장하여 반환

## 7. 가변적 은닉 계층 구성을 위한 역전파 함수 정의

In [6]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens
    
    hiddens = aux
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis = 0)
    
    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))): # 역전파를 위해 순서를 뒤집음
        G_hidden = G_hidden * relu_derv(hiddens[n+1]) # len(hiddens) =  len(pm_hiddens) + 1
        
        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis = 0)
        
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)
        
        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

## 8. 스위치 함수 정의

In [8]:
# 설정 정보의 존재 여부에 따라 앞서 정의된 두 묶음의 함수를 선택적으로 호출하는 세 개의 함수를 정의함
global hidden_config

def init_model():
    if hidden_config is not None:
        print('은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.'.format(len(hidden_config)))
        init_model_hiddens()
    else:
        print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.')
        init_model_hidden1()
        
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)

## 9. 은닉 계층 구조 지정 함수 정의

In [9]:
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
        
    else:
        hidden_config = info